In [4]:
# Import necessary libraries
import requests
import json
import pandas as pd
import folium

# Step 1: Define the Overpass API query to fetch petrol stations in Switzerland
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="CH"][admin_level=2];
node["amenity"="fuel"](area);
out body;
"""
# Fetch data from the Overpass API
response = requests.get(overpass_url, params={'data': overpass_query})
if response.status_code == 200:
    print("Data fetched successfully!")
    data = response.json()
else:
    print(f"Error fetching data: {response.status_code}")
    exit()

# Step 2: Save the data in .json format
json_file_path = "./data/petrol_stations.json"
with open(json_file_path, "w") as json_file:
    json.dump(data, json_file)
    print(f"Data saved to {json_file_path}")

# Step 3: Extract relevant data and save as .csv
# Extract nodes from the JSON response
elements = data.get("elements", [])
stations = []
for element in elements:
    if "tags" in element:
        tags = element["tags"]
        stations.append({
            "id": element.get("id"),
            "lat": element.get("lat"),
            "lon": element.get("lon"),
            "operator": tags.get("operator", "Unknown"),
            "brand": tags.get("brand", "Unknown")
        })

# Convert to a Pandas DataFrame
stations_df = pd.DataFrame(stations)

# Save to CSV
csv_file_path = "./data/petrol_stations.csv"
stations_df.to_csv(csv_file_path, index=False)
print(f"Data saved to {csv_file_path}")

# Step 4: Count the number of petrol stations per operator
pivot_table = stations_df.pivot_table(index="operator", aggfunc="size").sort_values(ascending=False)
print("Top 5 operators by number of petrol stations:")
print(pivot_table.head(5))

# Step 5: Create a map to visualize all petrol stations
# Initialize a Folium map centered on Switzerland
switzerland_map = folium.Map(location=[46.8182, 8.2275], zoom_start=8)

# Add petrol stations to the map
for _, row in stations_df.iterrows():
    popup_text = f"Brand: {row['brand']}<br>Operator: {row['operator']}"
    folium.Marker(
        location=[row["lat"], row["lon"]],
        popup=popup_text
    ).add_to(switzerland_map)

# Save the map to an HTML file
map_file_path = "./data/petrol_stations_map.html"
switzerland_map.save(map_file_path)
print(f"Map saved to {map_file_path}")

# Load the petrol stations data from the CSV file
csv_file_path = './data/petrol_stations.csv'
stations_df = pd.read_csv(csv_file_path)

# Initialize a Folium map centered on Switzerland
switzerland_map = folium.Map(location=[46.8182, 8.2275], zoom_start=8)

# Add petrol stations to the map
for _, row in stations_df.iterrows():
    popup_text = f"Brand: {row['brand']}<br>Operator: {row['operator']}"
    folium.Marker(
        location=[row["lat"], row["lon"]],
        popup=popup_text
    ).add_to(switzerland_map)

# Display the map inline in the notebook
switzerland_map

Data fetched successfully!
Data saved to ./data/petrol_stations.json
Data saved to ./data/petrol_stations.csv
Top 5 operators by number of petrol stations:
operator
Unknown            1645
Migrol              155
Coop Mineraloel      68
BP                   64
Schweizer Armee      61
dtype: int64
Map saved to ./data/petrol_stations_map.html
